In [1]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing as p
import pandas as pd

In [12]:
data_source = "https://archive.ics.uci.edu/ml/machine-learning-databases/restricted/breast-cancer/breast-cancer.data"

names = ["class", "age", "menopause", "tumor-size", "inv-nodes", "node-caps", "deg-malig", "breast", "breast-quad", "irradiat"]

df = pd.read_csv(data_source, names=names)
df[:5]
df.dtypes

class          object
age            object
menopause      object
tumor-size     object
inv-nodes      object
node-caps      object
deg-malig       int64
breast         object
breast-quad    object
irradiat       object
dtype: object

In [13]:
for column in range(0, 10):
    if column not in [6]:
        df[names[column]] = df[names[column]].astype("category")

df.dtypes

class          category
age            category
menopause      category
tumor-size     category
inv-nodes      category
node-caps      category
deg-malig         int64
breast         category
breast-quad    category
irradiat       category
dtype: object

In [14]:
y_data = df.pop("class")
y_data = pd.get_dummies(y_data).as_matrix()

x_data = df
x_data.insert(0, "bias", 1)
x_data = pd.get_dummies(x_data).as_matrix()

x_data

array([[ 1.,  3.,  0., ...,  0.,  1.,  0.],
       [ 1.,  2.,  0., ...,  1.,  1.,  0.],
       [ 1.,  2.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 1.,  1.,  0., ...,  0.,  1.,  0.],
       [ 1.,  3.,  0., ...,  0.,  1.,  0.],
       [ 1.,  3.,  0., ...,  0.,  1.,  0.]])

In [15]:
x_data = p.MinMaxScaler().fit_transform(x_data)

In [16]:
training_idx = np.random.randint(y_data.shape[0], size=int(y_data.shape[0] * 0.8))
test_idx = np.random.randint(y_data.shape[0], size=int(y_data.shape[0] * 0.2))

x_training, x_test = x_data[training_idx], x_data[test_idx]
y_training, y_test = y_data[training_idx], y_data[test_idx]

In [17]:
y_training.shape

(228, 2)

In [18]:
W = tf.Variable(tf.zeros([len(x_data[0]), len(y_data[0])]))
                
X = tf.placeholder("float", [None, len(x_data[0])])
Y = tf.placeholder("float", [None, len(y_data[0])])


hypothesis = tf.nn.softmax(tf.matmul(X, W))

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), reduction_indices=1))

a = tf.Variable(0.001)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

In [19]:
for step in range(500001):
    sess.run(train, feed_dict={X:x_training, Y:y_training})
    if step % 100000 == 0:
        print (step, sess.run(cost, feed_dict={X:x_training, Y:y_training}))

0 0.69275
100000 0.448247
200000 0.442815
300000 0.440633
400000 0.439319
500000 0.438405


In [22]:
h_values = sess.run(hypothesis,feed_dict={X:x_test})
result = [np.argmax(predict) == np.argmax(original_value) for predict, original_value in zip(h_values, y_test)]

sum(result)/len(y_test)

0.78947368421052633